# Train Mixed Tree

In [ ]:
import numpy as np
import pickle
import copy
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
import math


class mt:
    def pm(self, dataset, m, n, k=0):
        pxy = np.zeros((n, n, 4))
        l = 1
        for i in range(n):
            temp_dataset = dataset[dataset[:, i] == 0]
            for j in range(n):
                if pxy[i, j, 0] == 0:
                    pxy[i, j, 0] = (np.sum(temp_dataset[temp_dataset[:, j] == 0][:, n+k]))
                    count = temp_dataset[temp_dataset[:, j] == 0].shape[0]
                    l = pxy[i, j, 0]/count if count > 0 and pxy[i, j, 0]/count < l else l
                if pxy[j, i, 0] == 0:
                    pxy[j, i, 0] = pxy[i, j, 0]
                if pxy[i, j, 1] == 0:
                    pxy[i, j, 1] = (np.sum(temp_dataset[temp_dataset[:, j] == 1][:, n+k]))
                    count = temp_dataset[temp_dataset[:, j] == 1].shape[0]
                    l = pxy[i, j, 1]/count if count > 0 and pxy[i, j, 1]/count < l else l
                if pxy[j, i, 2] == 0:
                    pxy[j, i, 2] = pxy[i, j, 1]

            temp_dataset = dataset[dataset[:, i] == 1]
            for j in range(n):
                if pxy[i, j, 2] == 0:
                    pxy[i, j, 2] = (np.sum(temp_dataset[temp_dataset[:, j] == 0][:, n+k]))
                    count = temp_dataset[temp_dataset[:, j] == 0].shape[0]
                    l = pxy[i, j, 2]/count if count > 0 and pxy[i, j, 2]/count < l else l
                if pxy[j, i, 1] == 0:
                    pxy[j, i, 1] = pxy[i, j, 2]
                if pxy[i, j, 3] == 0:
                    pxy[i, j, 3] = (np.sum(temp_dataset[temp_dataset[:, j] == 1][:, n+k]))
                    count = temp_dataset[temp_dataset[:, j] == 1].shape[0]
                    l = pxy[i, j, 3]/count if count > 0 and pxy[i, j, 3]/count < l else l
                if pxy[j, i, 3] == 0:
                    pxy[j, i, 3] = pxy[i, j, 3]
        return (pxy+l)/(m+4*l)


    def mi(self, pxy, n):
        ixy = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i < j:
                    ixy[i, j] = pxy[i, j, 0]*np.log(pxy[i, j, 0]/(pxy[i, i, 0]*pxy[j, j, 0])) + pxy[i, j, 1]*np.log(pxy[i, j, 1]/(pxy[i, i, 0]*pxy[j, j, 3])) + \
                        pxy[i, j, 2]*np.log(pxy[i, j, 2]/(pxy[i, i, 3]*pxy[j, j, 0])) + \
                        pxy[i, j, 3]*np.log(pxy[i, j, 3]/(pxy[i, i, 3]*pxy[j, j, 3]))
        return ixy


    def dt(self, ewts, prnt=False, k=0, iter=0):
        ewts_cp = 1/copy.deepcopy(ewts)
        X = csr_matrix(ewts_cp)
        Tcsr = minimum_spanning_tree(X)
        e1 = [(item[0], item[1]) for item in np.transpose(np.nonzero(Tcsr.toarray()))]
        ntr1 = []
        mt.ctr(e1, ntr1, e1[0][0])
        return ntr1


    def ctr(self, ls, ntr, root):
        for ch in [item for item in ls if root in item]:
            if ch[0] == root:
                ntr.append(ch)
                ls.remove(ch)
                mt.ctr(ls, ntr, ch[1])
            else:
                ntr.append((ch[1], ch[0]))
                ls.remove(ch)
                mt.ctr(ls, ntr, ch[0])


    def em(self, dataset, tr, cols):
        rows = dataset.shape[0]
        exy = np.zeros((rows, len(tr), cols))
        for idx, ch in enumerate(tr):
            i, j = ch
            exy[:, idx, :] = np.hstack((((dataset[:, i] == 0) & (dataset[:, j] == 0)).astype(int).reshape(rows, 1), ((dataset[:, i] == 0) & (dataset[:, j] == 1)).astype(int).reshape(
                rows, 1), ((dataset[:, i] == 1) & (dataset[:, j] == 0)).astype(int).reshape(rows, 1), ((dataset[:, i] == 1) & (dataset[:, j] == 1)).astype(int).reshape(rows, 1)))
        return exy


    def MS(self, dataset, m, n, k, iter, prnt, pk):
        trs = []
        cps = []
        for ki in range(k):
            pxy = mt.pm(dataset, m, n, ki)
            ixy = mt.mi(pxy, n)
            tr = mt.dt(ixy, prnt, ki, iter)
            tr = [(tr[0][0], tr[0][0])] + tr
            trs.append(tr)
            cp = np.zeros((len(tr), pxy.shape[2]))
            for idx, ch in enumerate(tr):
                if ch[0] == ch[1]:
                    cp[idx] = pxy[ch[0], ch[1], :]
                else:
                    cp[idx] = np.hstack(((pxy[ch[0], ch[1], :2]/pxy[ch[0], ch[0], 0]),
                                         (pxy[ch[0], ch[1], 2:]/pxy[ch[0], ch[0], 3])))
            cps.append(cp)
        return trs, cps


    def random_init(self, dataset, m, n, k, iter, prnt):
        trs = []
        cps = []
        for ki in range(k):
            pxy = mt.pm(dataset[np.random.choice(m, 8, replace=False), :], m, n, ki)
            ixy = mt.mi(pxy, n)
            tr = mt.dt(ixy, prnt, ki, iter)
            tr = [(tr[0][0], tr[0][0])] + tr
            trs.append(tr)
            cp = np.zeros((len(tr), pxy.shape[2]))
            for idx, ch in enumerate(tr):
                if ch[0] == ch[1]:
                    cp[idx] = pxy[ch[0], ch[1], :]
                else:
                    cp[idx] = np.hstack(((pxy[ch[0], ch[1], :2]/pxy[ch[0], ch[0], 0]),
                                         (pxy[ch[0], ch[1], 2:]/pxy[ch[0], ch[0], 3])))
            cps.append(cp)
        return trs, cps


    def ES(self, dataset, m, n, k, trs, cps):
        ph = dataset[:, n:].sum(axis=0)/m
        wt_ij = np.zeros((m, k))
        min_ = np.full((m, 1), 10**-100)
        
        for j in range(k):
            wt_ij[:, j] = np.max(np.hstack(((ph[j] * np.sum(mt.em(dataset, trs[j], 4)*cps[j], axis=2).prod(axis=1)).reshape(m, 1), min_)), axis=1)

        wt_ij = wt_ij/np.sum(wt_ij, axis=1).reshape(m, 1)
        dataset[:, n:] = wt_ij
        
        pk = None
        return dataset, pk


    def train(self, dataset, m, n):
        k = 4
        print("k="+str(k))
        LL = []
        train = dataset
        for x in range(1):
            wt_ij = np.random.rand(m, k)
            wt_ij = wt_ij/np.sum(wt_ij, axis=1).reshape(m, 1)
            dataset = np.hstack((train, wt_ij))
            trs, cps = mt.random_init(dataset, m, n, k, 0, False)
            ph_1 = [0]*k

            for iter in range(1, 2):
                dataset, pk = mt.ES(dataset, m, n, k, trs, cps)
                trs, cps = mt.MS(dataset, m, n, k, iter, False, pk)
                ph_2 = dataset[:, n:].sum(axis=0)/m
                if (np.abs(ph_1 - ph_2) == 0.00000).all():
                    break
                ph_1 = ph_2

            lambda_k = dataset[:, n:].sum(axis=0)/m
            L = 0
        return trs, cps

    def generate_model(self, dataset, m, n):
        model = mt.train(dataset,m ,n)
        f = open('modelmt_book.pkl', 'wb')
        pickle.dump(model, f, -1)
        f.close() 


def dataset_load(file, file_2=None):
    dataset_1 = np.loadtxt(file, delimiter=",", dtype=int)
    if file_2:
        dataset_2 = np.loadtxt(file, delimiter=",", dtype=int)
        dataset = np.vstack((dataset_1, dataset_2))
    else:
        dataset = dataset_1
    return dataset, dataset.shape[0], dataset.shape[1]


mt = mt()
dataset, m, n = dataset_load("book.ts.data")
mt.generate_model(dataset,m,n)


k=4


<ipython-input-20-31e00e94f2fb>:58: RuntimeWarning: divide by zero encountered in true_divide
  ewts_cp = 1/copy.deepcopy(ewts)


# Testing

In [14]:
test = np.loadtxt("nltcs.valid.data", delimiter=",", dtype=int)
k=4
LL=[]
f = open('modelmt_nltcs.pkl', 'rb')   # 'rb' for reading binary file
trs, cps = pickle.load(f)     
f.close()
fin = np.zeros((test.shape[0],k))
for j in range(k):
    fin[:,j] = np.log(np.sum(mt.em(test, trs[j], 4)*cps[j], axis = 2)).sum(axis=1)
LL.append((fin.max(axis=1)).sum()/test.shape[0])

print("avgLL="+str(np.mean(LL)))


avgLL=-6.383167157453798


In [19]:
test = np.loadtxt("plants.valid.data", delimiter=",", dtype=int)
k=4
LL=[]
f = open('modelmt_plants.pkl', 'rb')   # 'rb' for reading binary file
trs, cps = pickle.load(f)     
f.close()
fin = np.zeros((test.shape[0],k))
for j in range(k):
    fin[:,j] = np.log(np.sum(mt.em(test, trs[j], 4)*cps[j], axis = 2)).sum(axis=1)
LL.append((fin.max(axis=1)).sum()/test.shape[0])

print("avgLL="+str(np.mean(LL)))

avgLL=-15.785542539306384


In [ ]:
test = np.loadtxt("book.valid.data", delimiter=",", dtype=int)
k=4
LL=[]
f = open('modelmt_book.pkl', 'rb')   # 'rb' for reading binary file
trs, cps = pickle.load(f)     
f.close()
fin = np.zeros((test.shape[0],k))
for j in range(k):
    fin[:,j] = np.log(np.sum(mt.em(test, trs[j], 4)*cps[j], axis = 2)).sum(axis=1)
LL.append((fin.max(axis=1)).sum()/test.shape[0])

print("avgLL="+str(np.mean(LL)))